In [16]:
from modelRoPE import *

basename = "shakespeare"

device = torch.device('cpu')

context_length = 128
embed_dim = 128
n_head = 4
n_layer = 2
batch_size = 64*4

vocab = Vocab()
model = LLM(vocab, context_length, embed_dim, n_head, n_layer).to(device)
model_c = torch.compile(model)
model.load_state_dict(torch.load("shakespeare1.pt"))
# model.load_state_dict(torch.load("harry_potter.pt"))

<All keys matched successfully>

# Generate

In [45]:
g = model.generate(torch.tensor(vocab.encode("\n"), dtype=torch.long)[None,:].to(device), 200, temperature=1.0, top_k=None)
print(vocab.decode(g[0].tolist()))


Did you never; send him time to question.

GRUMIO:
Takes for a bosom name of truth we see.

SecondUS:
I would not consul:
What Edward is all!

HORTENSIO:
I makeshepherd:
How fear!
There's'st a Earl of himself, custom say
Timogue had thirtyish'd with the moon, while your,
As dogs gentle supplell, the happy cheeks sinceThen.

LUDIO:
No, husband comes so, you shall staff to cruel
For what plain full arm I may injury,
Which makes some sweet Ext nature, we am.

JULIET:
Nor wiping, a name, AARTIUS:
Now, my am school-t Walk of mine excuse; unto
For, earnest, Trou art not to be done;

GLOUCESTER:
PSidius


## Export onnx

In [43]:
with torch.inference_mode():
    model.eval()
    from onnxruntime.quantization import quantize_dynamic
    device = torch.device('cpu')
    model = model.to(device)
    torch.onnx.export(
        model.cpu(),
        torch.zeros((0, context_length), dtype=torch.long),
        f'{basename}.onnx',
        export_params=True,
        opset_version=17,
        do_constant_folding=True,
        input_names=['input'],
        output_names=['output'],
        dynamic_axes={'input': {0: 'batch_size', 1: 'context'}, 'output': {0: 'batch_size', 1: 'context'}},
    )
    quantize_dynamic(f'{basename}.onnx', f'{basename}.8bit.onnx')

/usr/lib/python3.11/site-packages/torch/onnx/utils.py:516: FutureWarning: Caffe2 ONNX exporter is deprecated in version 2.0 and will be removed in 2.2. Please use PyTorch 2.1 or older for this capability.
  _export(
/home/etienne/Documents/Project/WebGPT/RoPE.py:47: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if self.cos_cached is not None and x.shape[-2] <= self.cos_cached.shape[-2] and start_idx == 0 :


Ignore MatMul due to non constant B: /[/blocks/blocks.0/mhsa/mhsa.1/MatMul]
Ignore MatMul due to non constant B: /[/blocks/blocks.0/mhsa/mhsa.1/MatMul_1]
Ignore MatMul due to non constant B: /[/blocks/blocks.1/mhsa/mhsa.1/MatMul]
Ignore MatMul due to non constant B: /[/blocks/blocks.1/mhsa/mhsa.1/MatMul_1]


## Import onnx

In [6]:
import onnx
model = onnx.load(f"{basename}.onnx")

# Check that the model is well formed
onnx.checker.check_model(model)

# Print a human readable representation of the graph
# print(onnx.helper.printable_graph(model.graph))


In [48]:
import onnxruntime as ort
import numpy as np
import scipy

ort_session = ort.InferenceSession(f"{basename}.onnx")

input = np.array(vocab.encode("I'll be a man, and I'll be a man."))[None,:]
# input = np.random.randint(0,len(vocab), size=128)[None,:]
outputs = ort_session.run(
    None,
    #{"input": np.random.randint(0,len(vocab),(batch_size, context_length))},
    {"input": input},
)
max = np.argmax(outputs,axis = -1)[0][0].tolist()
print(vocab.decode(max))

OL be a man. and I'll be a man.



# Test

In [ ]:
books = Books('harry_potter.txt')
val_set = BooksDataset(context_length, books, vocab, train=False)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False, drop_last=False, pin_memory=True, num_workers=8)
nll = model_c.evaluate_loss(val_loader, device)
print(f"---------------------")
print(f"Validation NLL {f'{nll:.2f}':>6}")
print(f"---------------------")

In [ ]:
import inspect
with open("logs.txt", "a") as file:
        file.write(f"model_file:{os.path.basename(inspect.getfile(LLM))}, eval_loss: {nll}\n")